# Data cleaning

In [ ]:
import pandas as pd
import glob
import os

DATA_FOLDER = "src\IEX_data"

def read_one_file(path):

    # read raw
    df = pd.read_excel(path, header=None)

    # find the actual header row (where "Date" appears)
    header_row = df[df.iloc[:,0] == "Date"].index[0]

    df = pd.read_excel(path, header=header_row)

    # remove summary section
    df = df[df["Time Block"].notna()]

    # keep only useful columns
    df = df[[
        "Date",
        "Time Block",
        "Purchase Bid (MW)",
        "Sell Bid (MW)",
        "Final Scheduled Volume (MW)",
        "MCP (Rs/MWh) *"
    ]]

    df.rename(columns={
        "Final Scheduled Volume (MW)": "volume",
        "MCP (Rs/MWh) *": "price",
        "Purchase Bid (MW)": "demand_bid",
        "Sell Bid (MW)": "supply_bid"
    }, inplace=True)

    return df

In [ ]:
all_files = glob.glob(os.path.join(DATA_FOLDER, "*.xlsx"))

frames = []

for f in all_files:
    print("Reading:", f)
    frames.append(read_one_file(f))

df = pd.concat(frames, ignore_index=True)


In [ ]:
df.head()

In [ ]:
df.index.is_monotonic_increasing


In [ ]:
df["time_start"] = df["Time Block"].str.split(" - ").str[0]
df["datetime"] = pd.to_datetime(df["Date"] + " " + df["time_start"], dayfirst=True)

df["hour"] = df["datetime"].dt.floor("H")


In [ ]:
len(df)

In [ ]:
df.to_csv("iex_data.csv", index=False)

In [6]:
import pandas as pd
import numpy as np
import glob
import re
from datetime import datetime
from pathlib import Path

DATA_FOLDER = "src\IEX_data"


# -------------------------------------------------------
# 1. Correct chronological ordering of monthly files
# -------------------------------------------------------
def extract_month_year(filename: str) -> datetime:
    """
    Extracts month and year from filename:
    DAM_Market Snapshot_<month>-<year>.xlsx

    Returns a datetime object (first day of that month)
    which allows proper chronological sorting.
    """

    # Example filename:
    # DAM_Market Snapshot_8-2024.xlsx

    match = re.search(r'(\d{1,2})-(\d{4})', filename)

    if not match:
        raise ValueError(f"Cannot extract month/year from: {filename}")

    month = int(match.group(1))
    year = int(match.group(2))

    return datetime(year, month, 1)


files = glob.glob(f"{DATA_FOLDER}/*.xlsx")
files = sorted(files, key=extract_month_year)

print("Files in chronological order:")
for f in files:
    print(Path(f).name)


# -------------------------------------------------------
# 2. Read one IEX Excel safely
# -------------------------------------------------------
def read_one_file(path):

    # read raw sheet first (no header)
    raw = pd.read_excel(path, header=None, engine="openpyxl")

    # find true header row containing "Time Block"
    header_row = None
    for i in range(len(raw)):
        row = raw.iloc[i].astype(str).str.lower()
        if row.str.contains("time block").any():
            header_row = i
            break

    if header_row is None:
        raise Exception(f"Could not find data table in {path}")

    # reload with proper header
    df = pd.read_excel(path, header=header_row, engine="openpyxl")

    # drop completely empty rows
    df = df.dropna(how="all")

    # keep only valid auction rows (15-minute time block)
    mask = df["Time Block"].astype(str).str.match(r"\d{2}:\d{2}\s-\s\d{2}:\d{2}")
    df = df[mask].copy()

    # IEX only writes date once per day → forward fill
    df["Date"] = df["Date"].ffill()

    # keep only relevant double-auction information
    df = df[[
        "Date",
        "Time Block",
        "Purchase Bid (MW)",
        "Sell Bid (MW)",
        "Final Scheduled Volume (MW)",
        "MCP (Rs/MWh) *"
    ]]

    # rename for sanity
    df.rename(columns={
        "Purchase Bid (MW)": "demand_bid",
        "Sell Bid (MW)": "supply_bid",
        "Final Scheduled Volume (MW)": "volume",
        "MCP (Rs/MWh) *": "price"
    }, inplace=True)

    # convert numeric columns
    for col in ["demand_bid", "supply_bid", "volume", "price"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df = df.dropna(subset=["price", "volume"])

    # extract start time from "HH:MM - HH:MM"
    df["time_start"] = df["Time Block"].str.split("-").str[0].str.strip()

    # fix 24:00 bug
    df["time_start"] = df["time_start"].replace("24:00", "00:00")

    # create timestamp
    df["datetime"] = pd.to_datetime(
        df["Date"].astype(str) + " " + df["time_start"],
        format="%d-%m-%Y %H:%M",
        dayfirst=True,
        errors="raise"
    )

    return df[["datetime", "demand_bid", "supply_bid", "volume", "price"]]


# -------------------------------------------------------
# 3. Combine all months
# -------------------------------------------------------
frames = []

for f in files:
    print("Parsing:", Path(f).name)
    frames.append(read_one_file(f))

df = pd.concat(frames, ignore_index=True)


# -------------------------------------------------------
# 4. Sort and set time index
# -------------------------------------------------------
df = df.sort_values("datetime")
df = df.set_index("datetime")


# -------------------------------------------------------
# 5. Integrity Checks (VERY IMPORTANT)
# -------------------------------------------------------
print("\nDATA VALIDATION")

print("Monotonic time:", df.index.is_monotonic_increasing)
print("Unique timestamps:", df.index.is_unique)

daily_counts = df.resample("D").size()
print("Auctions per day (should be 96):")
print(daily_counts.value_counts())


# -------------------------------------------------------
# 6. Convert 15-minute auctions → hourly VWAP
# -------------------------------------------------------
def vwap(group):
    return (group["price"] * group["volume"]).sum() / group["volume"].sum()

hourly = df.resample("h").agg({
    "volume": "sum",
    "demand_bid": "mean",
    "supply_bid": "mean"
})

hourly["price"] = df.groupby(pd.Grouper(freq="h")).apply(vwap)

hourly = hourly.dropna()


# -------------------------------------------------------
# 7. Save outputs
# -------------------------------------------------------
df.reset_index().to_csv("iex_15min_market.csv", index=False)
hourly.reset_index().to_csv("iex_hourly_market.csv", index=False)

print("\nDONE.")
print("15-min auctions saved → iex_15min_market.csv")
print("Hourly market saved → iex_hourly_market.csv")


Files in chronological order:
DAM_Market Snapshot_1-2024.xlsx
DAM_Market Snapshot_2-2024.xlsx
DAM_Market Snapshot_3-2024.xlsx
DAM_Market Snapshot_4-2024.xlsx
DAM_Market Snapshot_5-2024.xlsx
DAM_Market Snapshot_6-2024.xlsx
DAM_Market Snapshot_7-2024.xlsx
DAM_Market Snapshot_8-2024.xlsx
DAM_Market Snapshot_9-2024.xlsx
DAM_Market Snapshot_10-2024.xlsx
DAM_Market Snapshot_11-2024.xlsx
DAM_Market Snapshot_12-2024.xlsx
DAM_Market Snapshot_1-2025.xlsx
DAM_Market Snapshot_2-2025.xlsx
DAM_Market Snapshot_3-2025.xlsx
DAM_Market Snapshot_4-2025.xlsx
DAM_Market Snapshot_5-2025.xlsx
DAM_Market Snapshot_6-2025.xlsx
DAM_Market Snapshot_7-2025.xlsx
DAM_Market Snapshot_8-2025.xlsx
DAM_Market Snapshot_9-2025.xlsx
DAM_Market Snapshot_10-2025.xlsx
DAM_Market Snapshot_11-2025.xlsx
DAM_Market Snapshot_12-2025.xlsx
Parsing: DAM_Market Snapshot_1-2024.xlsx
Parsing: DAM_Market Snapshot_2-2024.xlsx
Parsing: DAM_Market Snapshot_3-2024.xlsx
Parsing: DAM_Market Snapshot_4-2024.xlsx
Parsing: DAM_Market Snapshot_5-2

# Data reading

In [1]:
import csv
from pathlib import Path
from datetime import datetime, timedelta

In [2]:
time_del = timedelta(hours=1)
IEX_DATA_DIR = Path(r'C:\Users\Abhyuday Chauhan\projects\FY_project\src\fy_project\IEX_data\iex_hourly_market.csv')

In [3]:
class IEXMarket:
    def __init__(self):
        self.reader = csv.DictReader(open(IEX_DATA_DIR, "r"))

    def get_price(self, dt: datetime) -> float:
        try:
            val = next(self.reader)
        except StopIteration:
            self.reader = csv.DictReader(open(IEX_DATA_DIR, "r"))
            print("\n\n Hit end of file, looping back to start \n\n")
            val = next(self.reader)

        date_str = val["datetime"]
        if date_str != dt.strftime("%Y-%m-%d %H:%M:%S"):
            raise IndexError(
                f"Expected datetime {dt} not found in IEX data. Got {date_str} instead."
            )

        return float(val["price"]) if "price" in val else 10.0

In [8]:
date = datetime(2024, 1, 1, 0, 0)
market = IEXMarket()

In [38]:

print(market.get_price(date))
date =date + time_del

3100.3716789506652
